In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [7]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [8]:

# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.214s

OK


In [9]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [10]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [11]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [16]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.165s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [38]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, TxFetcher


# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4

prev_tx = TxFetcher.fetch('2d787742f2d93c633e106211689d775be1b3d35a70a20f982c7b41600857691d', testnet=True)
print(prev_tx)
print(decode_base58('mr4oRvrU2CiNrYJw7adbfpS6mYEVoNLAYR').hex())
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_in = TxIn(
    bytes.fromhex('2d787742f2d93c633e106211689d775be1b3d35a70a20f982c7b41600857691d'),
    1
)

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee

# create a transaction output for the target amount and address
tx_out_target = TxOut(
    1000,
    p2pkh_script(decode_base58(target_address))
)

# create a transaction output for the change amount and address

tx_out_change = TxOut(
    100,
    p2pkh_script(decode_base58('mr4oRvrU2CiNrYJw7adbfpS6mYEVoNLAYR'))
)


# create the transaction object
tx = Tx(
    1,
    [tx_in],
    [tx_out_target, tx_out_change],
    0,
    testnet=True
)

passphrase = b'your@email.address H998wFCmmQxn'
secret = little_endian_to_int(hash256(passphrase))

pk = PrivateKey(secret)
print(pk.point.address(testnet=True, compressed=True))

# sign the one input in the transaction object using the private key
tx.sign_input(0, pk)

# print the transaction's serialization in hex

print(tx.serialize().hex())
# broadcast at http://testnet.blockchain.info/pushtx

tx: 2d787742f2d93c633e106211689d775be1b3d35a70a20f982c7b41600857691d
version: 2
tx_ins:
2765adc03a6d5d861a18887325d8118c02155ed7a0e1bfd6c67e2199bb7e4cee:1
tx_outs:
1609625:OP_HASH160 67da09f581e4a847da0f875376cc38d2d60922b5 OP_EQUAL
10000:OP_DUP OP_HASH160 73b72b4c99ad42e8f2d77363197a7f8d4a09b403 OP_EQUALVERIFY OP_CHECKSIG
locktime: 1607743
73b72b4c99ad42e8f2d77363197a7f8d4a09b403
mr4oRvrU2CiNrYJw7adbfpS6mYEVoNLAYR
01000000011d69570860417b2c980fa2705ad3b3e15b779d681162103e633cd9f24277782d010000006a473044022036a09d66799b911b78a4b5905f5e88d61ce41a044f9fb333e31a7cbbf745c7a602206842982a95968a34d0765962bfb72b24cb718b338ce469b7401c975883e3dc330121020a75f60d84b35ef39ca8ae8114281873ce8209cc8a3e1905a02046222003b38dffffffff02e8030000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac64000000000000001976a91473b72b4c99ad42e8f2d77363197a7f8d4a09b40388ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [39]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got

prev_tx = TxFetcher.fetch('ddcfa3f132356f2eebafc49625a48f3c31e699e327744f52805c4e98638130fa', testnet=True)
print(prev_tx)
print(decode_base58('mr4oRvrU2CiNrYJw7adbfpS6mYEVoNLAYR').hex())
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence

# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
tx_in_change = TxIn(
    bytes.fromhex('ddcfa3f132356f2eebafc49625a48f3c31e699e327744f52805c4e98638130fa'),
    1
)

# create the second transaction input with the default ScriptSig and
# sequence
tx_in_faucet = TxIn(
    bytes.fromhex('c5abf92d3b1afc1a1339e680fe3054936b6fd22a92c014115e207e6ff4d10c38'),
    0
)

# create a transaction output for the amount and address
# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
tx_out = TxOut(
    1000,
    p2pkh_script(decode_base58('mr4oRvrU2CiNrYJw7adbfpS6mYEVoNLAYR'))
)

tx = Tx(
    1,
    [tx_in_change, tx_in_faucet],
    [tx_out],
    0,
    testnet=True
)

# sign the first input using the private key
tx.sign_input(0, pk)
# sign the second input using the private key 
tx.sign_input(1, pk)

# print the transaction's serialization in hex
print(tx.serialize().hex())




# broadcast at http://testnet.blockchain.info/pushtx

tx: ddcfa3f132356f2eebafc49625a48f3c31e699e327744f52805c4e98638130fa
version: 1
tx_ins:
2d787742f2d93c633e106211689d775be1b3d35a70a20f982c7b41600857691d:1
tx_outs:
6000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
1000:OP_DUP OP_HASH160 73b72b4c99ad42e8f2d77363197a7f8d4a09b403 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
73b72b4c99ad42e8f2d77363197a7f8d4a09b403
0100000002fa308163984e5c80524f7427e399e6313c8fa42596c4afeb2e6f3532f1a3cfdd010000006b48304502210088c3c2b640a8ce76c3c776b4453bc68a32f0a74ea1b7f41756608cb7663a5ba502201f333229b696cd92a7e2b9d295458146c285d5bebb46dad23d9fb008b2fd872e0121020a75f60d84b35ef39ca8ae8114281873ce8209cc8a3e1905a02046222003b38dffffffff380cd1f46f7e205e1114c0922ad26f6b935430fe80e639131afc1a3b2df9abc5000000006b483045022100a0dc5b7c0933265ca67feed003b30bbc1e552084c3d8540565598ab7d7c8ce7f02202790ce5b9566af259a7fb2c37b99257653764cf45292561c3b7cf76ec11f49f50121020a75f60d84b35ef39ca8ae8114281873ce8209cc8a3e1905a02046222003b38dfffffff